In [1]:
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats

from cbsim.net import Net
from cbsim.request import Request
from cbsim.stochastic import Stochastic
from cbsim.experiment import read_exper_data
from cbsim.experiment import equal_var_test
from cbsim.experiment import pooled_var_t_test
from cbsim.experiment import separate_var_t_test

In [12]:
net = Net()
net.load_from_file(fnodes='nodes.txt', flinks='links.txt')

# traffic flow studies from May 2021
days = 5
fdata = {
    0: [[68, 70, 92, 84, 119], [70, 82, 80, 67, 77]],           # Sławkowska
    2: [[52, 38, 35, 40, 54], [45, 38, 48, 40, 40]],            # Szczepańska
    4: [[67, 64, 113, 129, 131], [149, 157, 142, 142, 171]],    # Św. Anny
    5: [[30, 27, 31, 31, 43], [66, 64, 60, 55, 93]],            # Franciszkańska
    6: [[22, 38, 36, 35, 52], [149, 134, 120, 184, 102]],       # Poselska
    10: [[153, 143, 172, 176, 214], [130, 122, 119, 152, 142]], # Sienna
    12: [[43, 45, 49, 57, 60], [57, 48, 57, 59, 69]]            # Szpitalna
}

flows = {}
for f in fdata:
    d = fdata[f]
    flows[f] = int(np.array([d[0][i] + d[1][i] for i in range(days)]).max())
_total = sum(flows.values()) # total number of requests to generate
print(_total)

loadpoints = {15: 'A', 28: 'B', 24: 'C', 19: 'D', 8: 'E', 20: 'F'}
probs = { 'R': 0.2, 'H': 0.05, 'S': 0.2, 'P': 0.15, 'W': 0.05 }
# probs = { 'R': 1, 'H': 1, 'S': 1, 'P': 1, 'W': 1 }

clients = [nd for nd in net.nodes if nd.type != 'N' and nd.type != 'L']
entries = sorted(flows.keys())
outlets = [
    0,  # Sławkowska
    1,  # Św. Tomasza
    3,  # Szewska
    5,  # Franciszkańska
    7,  # Kanonicza
    8,  # Grodzka
    9,  # Dominikańska
    10, # Sienna
    11, # Mikołajska
    12  # Szpitalna
]
zones = sorted([r.code for r in net.regions])

def simulate(weight=0.15, capacity=0.15):

    # 1) generate requests
    generated = 0
    reqs = []
    while generated < _total:
        dst = np.random.choice(clients)
        if np.random.random() < probs[dst.type]:
            req = Request(0, None, dst)
            generated += 1
            reqs.append(req)
    # 2) generate demand using the proposed aproach
    reqs = net.gen_demand(flows=flows,
                        probs=probs,
                        requests=reqs,
                        s_weight=Stochastic(law=1, location=weight, scale=0.3*weight),
                        traditional=False)
    # 3) calculate distances savings
    dds = net.simulate(requests=reqs, outlets=outlets, loadpoints=None, capacity=capacity)
    bds = net.simulate(requests=reqs, outlets=outlets, loadpoints=loadpoints.keys(), capacity=capacity)
    # print(len(dds[0]), len(dds[1]), sum(dds[0]), sum(dds[1]))
    # print(len(bds[0]), len(bds[1]), sum(bds[0]), sum(bds[1]), sum(bds[0]) + sum(bds[1]))
    #print(sum(dds[0]) - sum(bds[0]) - sum(bds[1]))
    #return dds[0], bds[0], bds[1]
    return sum(dds[0]) - sum(bds[0]) - sum(bds[1])


1435


<H1>Run simulations

In [13]:
[simulate() for _ in range(50)]

[278.4569999999971,
 291.8729999999984,
 272.1659999999956,
 301.7919999999969,
 298.09999999999417,
 267.4819999999953,
 287.1819999999972,
 295.76399999999614,
 290.18599999999805,
 290.20999999999424,
 290.6239999999964,
 282.24399999999576,
 281.18199999999666,
 268.5469999999982,
 282.87399999999536,
 281.8489999999979,
 279.08999999999605,
 281.17799999999727,
 276.5489999999966,
 273.3889999999973,
 285.83699999999703,
 288.73999999999495,
 277.4129999999949,
 249.6629999999979,
 283.9819999999986,
 284.397999999995,
 279.43799999999516,
 284.16199999999594,
 280.2740000000004,
 288.736999999999,
 276.77099999999587,
 283.76599999999695,
 276.7199999999939,
 269.9189999999949,
 292.3209999999964,
 286.188999999996,
 282.9259999999952,
 283.7089999999978,
 280.55499999999756,
 268.5229999999948,
 295.38399999999564,
 281.32699999999727,
 284.34299999999627,
 289.11499999999774,
 287.27099999999865,
 295.3109999999951,
 291.4389999999936,
 285.66599999999534,
 273.18199999999615,


In [2]:
# Mean flows
ds1 = [235.64900000000108,
 232.36600000000277,
 235.74400000000128,
 253.4190000000013,
 251.3310000000028,
 238.00000000000205,
 233.16800000000217,
 220.28600000000267,
 238.71300000000286,
 234.333999999998,
 233.05900000000202,
 248.6469999999989,
 246.33400000000216,
 233.15999999999798,
 223.03299999999751,
 241.85400000000033,
 230.2580000000027,
 230.72100000000228,
 228.45100000000127,
 243.4320000000011,
 255.5860000000008,
 242.07200000000324,
 253.01200000000023,
 244.16000000000093,
 235.86600000000107,
 245.08999999999878,
 232.26200000000273,
 223.89200000000153,
 226.49400000000253,
 223.5190000000033,
 238.4810000000008,
 227.6199999999996,
 233.16600000000136,
 237.91900000000146,
 236.3320000000008,
 247.64900000000011,
 251.31599999999997,
 238.8959999999998,
 243.30200000000133,
 238.78300000000132,
 243.0530000000016,
 235.79900000000254,
 238.41399999999993,
 215.70700000000215,
 240.31800000000192,
 235.2480000000009,
 223.74500000000256,
 226.61300000000045,
 241.6240000000018,
 224.28600000000125]
ds0 = [244.58600000000274,
 241.66200000000026,
 227.77900000000113,
 233.01500000000067,
 241.29600000000033,
 224.29300000000183,
 246.22300000000217,
 249.76799999999764,
 231.20399999999995,
 233.15600000000296,
 238.21000000000112,
 242.1770000000032,
 225.31700000000382,
 235.68100000000067,
 244.4300000000004,
 237.85100000000062,
 253.4919999999991,
 231.5640000000015,
 237.53600000000012,
 233.53000000000196,
 245.5510000000001,
 264.1879999999987,
 241.81600000000026,
 220.99000000000228,
 220.62500000000006,
 254.0250000000015,
 242.88600000000224,
 228.128000000003,
 225.5940000000017,
 224.48200000000162,
 229.44800000000055,
 247.09200000000203,
 222.10500000000224,
 241.87200000000223,
 240.31800000000226,
 216.10700000000213,
 240.0530000000017,
 233.7920000000048,
 222.03700000000225,
 240.8340000000029,
 229.4050000000001,
 239.95400000000217,
 219.30400000000301,
 232.5980000000012,
 239.25400000000116,
 233.4280000000009,
 226.90700000000055,
 216.0290000000029,
 243.45100000000082,
 249.3020000000006]

In [4]:
equal_var_test(ds0, ds1)

{'smaller': '2nd argument',
 'df1': 49,
 'df2': 49,
 'var1': 108.36408052998891,
 'var2': 82.93400666439564,
 'Fe': 1.3066302339461269,
 'Ft': 1.7621885352431106,
 'result': True}

In [5]:
pooled_var_t_test(ds0, ds1)

{'xmean': 235.6869000000014,
 'ymean': 236.6436600000012,
 'xvar': 108.36408052998891,
 'yvar': 82.93400666439564,
 'pvar': 95.64904359719228,
 'df': 98,
 't': 0.48913945864085634,
 'talpha': 1.984467454426692,
 'result': True}

In [7]:
# Min flows
ds1 = [198.34900000000172,
 198.5240000000034,
 205.48200000000298,
 214.7450000000019,
 212.45800000000358,
 220.98800000000244,
 188.66200000000273,
 204.68500000000333,
 197.4730000000043,
 218.25900000000408,
 228.00500000000443,
 199.59800000000428,
 211.90400000000338,
 210.50300000000215,
 215.50400000000283,
 216.51800000000387,
 196.3300000000021,
 207.75400000000334,
 204.2550000000029,
 199.8440000000027,
 215.07700000000347,
 206.30500000000418,
 207.02200000000133,
 203.5340000000026,
 208.07900000000535,
 198.32500000000232,
 210.34100000000245,
 203.5090000000025,
 206.25900000000385,
 207.53800000000152,
 219.33000000000186,
 216.10300000000268,
 200.57800000000293,
 190.78800000000288,
 216.474000000004,
 215.3930000000036,
 214.24600000000106,
 225.43600000000401,
 200.62500000000176,
 196.47800000000197,
 193.35200000000327,
 223.74500000000268,
 203.36500000000282,
 205.7140000000023,
 221.52100000000314,
 196.85300000000223,
 202.94100000000296,
 223.22400000000374,
 220.84400000000318,
 206.95800000000312]
ds0 = [191.76100000000304,
 206.67700000000454,
 211.28500000000116,
 213.1470000000022,
 202.66500000000366,
 214.56600000000407,
 202.69100000000284,
 223.13800000000356,
 211.05400000000273,
 208.37000000000472,
 205.73900000000214,
 200.56300000000246,
 200.99200000000363,
 199.73900000000148,
 202.48100000000298,
 188.36200000000437,
 192.06600000000242,
 211.81700000000276,
 205.1170000000024,
 190.83500000000166,
 212.63600000000204,
 201.80500000000157,
 198.166000000003,
 195.9400000000035,
 211.13000000000196,
 207.39000000000289,
 200.51300000000256,
 220.69600000000452,
 199.56300000000292,
 203.88100000000296,
 201.19500000000275,
 203.30200000000195,
 203.51100000000272,
 204.1310000000029,
 216.85300000000314,
 208.34700000000336,
 218.1170000000031,
 207.27500000000347,
 206.68200000000317,
 206.57600000000303,
 207.65500000000222,
 207.06700000000322,
 201.1470000000026,
 206.1810000000015,
 208.89300000000372,
 199.68800000000164,
 200.27700000000345,
 197.76100000000332,
 198.82400000000305,
 207.15400000000176]

In [8]:
equal_var_test(ds0, ds1)

{'smaller': '1st argument',
 'df1': 49,
 'df2': 49,
 'var1': 89.5685149364036,
 'var2': 52.1058930836015,
 'Fe': 1.7189709193295095,
 'Ft': 1.7621885352431106,
 'result': True}

In [9]:
pooled_var_t_test(ds0, ds1)

{'xmean': 204.90842000000285,
 'ymean': 208.19594000000296,
 'xvar': 52.1058930836015,
 'yvar': 89.5685149364036,
 'pvar': 70.83720401000257,
 'df': 98,
 't': 1.9530245496570902,
 'talpha': 1.984467454426692,
 'result': True}

In [2]:
# Max flows
ds1 = [268.05999999999494,
 285.3219999999986,
 262.997999999995,
 254.68299999999635,
 277.98199999999537,
 276.7739999999972,
 285.9829999999936,
 283.1439999999965,
 272.47299999999416,
 281.00399999999894,
 281.5179999999937,
 268.46899999999766,
 291.55299999999755,
 292.3249999999935,
 298.4769999999962,
 303.45399999999546,
 269.97699999999895,
 270.41299999999507,
 261.43699999999757,
 290.675999999995,
 282.91399999999385,
 279.6339999999999,
 295.19299999999566,
 293.24599999999816,
 293.2539999999961,
 258.86999999999597,
 274.00799999999725,
 265.4089999999963,
 291.644999999996,
 292.3999999999962,
 286.2619999999969,
 279.0159999999963,
 286.1019999999936,
 280.5719999999992,
 293.95699999999505,
 298.285999999997,
 301.4889999999965,
 270.6849999999956,
 295.690999999993,
 268.60199999999816,
 282.83399999999773,
 292.1759999999965,
 287.4179999999966,
 289.6359999999964,
 266.94099999999787,
 298.7219999999961,
 299.00799999999776,
 284.70499999999595,
 293.3989999999983,
 270.82799999999526]
ds0 = [278.4569999999971,
 291.8729999999984,
 272.1659999999956,
 301.7919999999969,
 298.09999999999417,
 267.4819999999953,
 287.1819999999972,
 295.76399999999614,
 290.18599999999805,
 290.20999999999424,
 290.6239999999964,
 282.24399999999576,
 281.18199999999666,
 268.5469999999982,
 282.87399999999536,
 281.8489999999979,
 279.08999999999605,
 281.17799999999727,
 276.5489999999966,
 273.3889999999973,
 285.83699999999703,
 288.73999999999495,
 277.4129999999949,
 249.6629999999979,
 283.9819999999986,
 284.397999999995,
 279.43799999999516,
 284.16199999999594,
 280.2740000000004,
 288.736999999999,
 276.77099999999587,
 283.76599999999695,
 276.7199999999939,
 269.9189999999949,
 292.3209999999964,
 286.188999999996,
 282.9259999999952,
 283.7089999999978,
 280.55499999999756,
 268.5229999999948,
 295.38399999999564,
 281.32699999999727,
 284.34299999999627,
 289.11499999999774,
 287.27099999999865,
 295.3109999999951,
 291.4389999999936,
 285.66599999999534,
 273.18199999999615,
 289.46699999999595]

In [3]:
equal_var_test(ds0, ds1)

{'smaller': '1st argument',
 'df1': 49,
 'df2': 49,
 'var1': 148.49750768959618,
 'var2': 82.66079320159824,
 'Fe': 1.7964684578749601,
 'Ft': 1.7621885352431106,
 'result': False}

In [4]:
separate_var_t_test(ds0, ds1)

21.373664028361592 0.23579033896113522


{'xmean': 282.9457199999964,
 'ymean': 282.5924799999963,
 'xvar': 82.66079320159824,
 'yvar': 148.49750768959618,
 't': 0.16428583955045822,
 'df': 91,
 'talpha': 1.9863771543000648,
 'result': True}

In [5]:
pooled_var_t_test(ds0, ds1)

{'xmean': 282.9457199999964,
 'ymean': 282.5924799999963,
 'xvar': 82.66079320159824,
 'yvar': 148.49750768959618,
 'pvar': 115.57915044559721,
 'df': 98,
 't': 0.16428583955045822,
 'talpha': 1.984467454426692,
 'result': True}